In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../MEGA/Data/Kinect trials/2014-12-22_P007_Pre_004.txt', skiprows=range(22),\
                header=None, names = [i for i in range(-2, 60)], sep='\t', engine='python')

# Change some column names
df.rename(columns={-2: 'Frame', -1: 'Part'}, inplace=True)

In [3]:
df_HEAD = df[df['Part'] == 'HEAD']

In [4]:
# Replace any strings with nan in the Frame column
df['Frame'] = df['Frame'].replace(r'[^0-9]', np.nan, regex=True)

In [5]:
# Convert the strings in the frame column to numbers
df['Frame'] = pd.to_numeric(df['Frame'])

In [6]:
n_frames= max(df['Frame'])

In [7]:
frame_624 = df[df['Frame'] == 624]

In [8]:
x = frame_624.as_matrix()

In [9]:
# Part names
parts = df.groupby('Part').groups.keys()

In [10]:

# Dataframe for current image frame
frame_df = frame_624


In [11]:
max_num_coords = 60
part = 'R_HAND'

def read_positions(frame_df, part, max_num_coords):
    
    row = frame_df[frame_df['Part'] == part]
    row_coordinates = row.loc[:, range(max_num_coords)].as_matrix()

    n_coordinates = np.sum(~np.isnan(row_coordinates))
    n_points = int(n_coordinates / 3)
    
    points = np.full([n_points, 3], np.nan)

    for idx, i in enumerate(range(0, n_coordinates, 3)):
        points[idx, :] = row_coordinates[0, i:i+3]
        
    return points    
        
population_dict = {part: read_positions(frame_df, part, max_num_coords) for part in parts}

In [12]:
part_types = ['HEAD', 'HIP', 'UPPER_LEG', 'KNEE', 'LOWER_LEG', 'FOOT']

In [13]:

def get_population(population_dict, part_types):

    population_list, labels = [], []

    for i, part_type in enumerate(part_types):
        for full_part_name in population_dict:

            if part_type in full_part_name:
                points = population_dict[full_part_name]
                n_points, _ = points.shape

                population_list.append(points)
                labels.extend([i for _ in range(n_points)])

    # Convert list to numpy matrix            
    population = np.concatenate(population_list)

    assert(population.shape[0] == len(labels))
    
    return population, labels

population, labels = get_population(population_dict, part_types)

[0, 0]
[0, 0, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3,

In [14]:
from scipy.spatial.distance import cdist

dist_matrix = cdist(population, population)

In [15]:
n_nodes = len(labels)


In [16]:

lengths = [63.9626,   19.3718,   12.8402,   22.0421,   20.5768]

edges = np.matrix('0 1;  \
                   1 2;  \
                   1 3;  \
                   2 3;  \
                   3 4;  \
                   3 5;  \
                   4 5')


In [17]:

def lengths_lookup(edges):

    last_part = edges.max()

    expected_lengths = {i: {} for i in range(last_part+1)}

    n_rows = len(edges)

    for i in range(n_rows):
            u, v = edges[i, 0], edges[i, 1]

            expected_lengths[u][v] = sum(lengths[u:v])
    
    return expected_lengths

    
expected_lengths = lengths_lookup(edges)

In [18]:
def adj_list_to_matrix(G):
    
    n_nodes = len(G)
    M = np.full((n_nodes, n_nodes), np.nan)
    
    for u in G:
        for v in G[u]:
            M[u, v] = G[u][v]
        
    return M


def adj_matrix_to_list(M):
    
    n_nodes = len(M)
    G = {i: {} for i in range(n_nodes)}
    
    for u in range(n_nodes):
        for v in range(n_nodes):
            G[u][v] = M[u, v]
    
    return G

In [19]:

def distances_to_adj_matrix(dist_matrix, labels, expected_lengths, cost_func):

    n_points = len(dist_matrix)
    
    M = np.full((n_nodes, n_nodes), np.nan)

    for i in range(n_points):
        label_A = labels[i]

        for j in range(n_points):
            label_B = labels[j]

            if label_B in expected_lengths[label_A]:
                expected_length = expected_lengths[label_A][label_B]
                measured_length = dist_matrix[i, j]

                M[i, j] = cost_func(expected_length, measured_length)
           
    return M
            
def norm_ratio(a, b):
    if a <= b:
        return a / b
    else:
        return b / a
    
cost_func = lambda a, b: (a - b)**2
    
M = distances_to_adj_matrix(dist_matrix, labels, expected_lengths, cost_func)

ratio_matrix = distances_to_adj_matrix(dist_matrix, labels, expected_lengths, norm_ratio)

In [25]:
score_func = lambda x : -(x - 1)**2 + 1
score_matrix = score_func(ratio_matrix)
